In [ ]:
import sys
stdout = sys.stdout
import numpy as np
from numpy.random import shuffle
sys.stdout = stdout
import time

In [ ]:
class Network(object):
    def __init__(
        self,
        nodes,
        step_size=0.0000000000001,
        lmbda=0.001
    ):
        """
        nodes represents the number of nodes per layer. 
        eg: [2,3,5] is 2 feature input, 
        3 neurons in the first layer,
        5 neurons in the final layer.
        5 also represents the number of classes
        
        nb_layers includes both the output and input layers
        weights contains the weights between each subsequent layer.
        biases stores the biases with the same indexing as weights
        outputs is the output matrix of each layer. An output matrix is of shape (p, n),
            where p is the number of examples given to the feedforward, and n is the nb of nodes in the layer
        gradients is the derivates of each layer. Each row is a different layer
        deltas stores the update values for each layer
        lmbda is the L2 regularization parameter.
        """
        self.step_size = step_size
        
        self.nb_layers = len(nodes)
        self.nodes = nodes
        self.weights = [ np.zeros((n, m)) for n,m in zip(nodes[:-1], nodes[1:])]
        for weight in self.weights:
            weight.fill(0.001)
        self.biases = [np.random.rand(1,n) for n in self.nodes[1:]]
        self.outputs = [ 0 for n in nodes[1:]]
        self.error_ds = np.zeros(nodes[-1])
        self.gradients = [ 0 for n in nodes[1:]]
        self.deltas = [ 0 for n in nodes[1: ]]
        self.lmbda = lmbda
    
    def fforw(self, inputs, labels):
        curr_input = inputs
        for w in range(self.nb_layers-1):
#             print(type(curr_input), type(self.weights[w]))
#             print("inp", curr_input)
#             print("weights", self.weights[w] )
            next_inp = np.matmul(curr_input, self.weights[w])
#             print("inp . weights", next_inp)
            next_inp += self.biases[w]
#             print(trans_bias.shape, self.biases[w].shape)
#             print(next_inp.shape)
            next_act = self.sigmoid(next_inp)
#             print("after adding bias", next_inp)
#             print("applied sigmoid", next_act)
            self.outputs[w] = next_act
            curr_input = next_act
    
    def backprop(self, inputs, labels):
        # using MSE loss, calculate error
        err = np.zeros((len(labels), self.nodes[-1]), dtype=float)
        for i,label in enumerate(labels):
            expected = int(label[0])
            err[i][expected] = label[0]
#         print(len(self.outputs))
#         print(err)
        err = err - self.outputs[-1]
#         print(err)
#         print(self.outputs[-1])
#         print(err.shape)
        # calculate gradients
        for i in range(self.nb_layers-1):
            self.gradients[i] = self.sigmoid(self.outputs[i], True)
        
        # calculate delta
        self.deltas[self.nb_layers-2] = err * self.gradients[self.nb_layers-2]
        
        for i in reversed(range(0, self.nb_layers-2)):
#             print(self.deltas[i+1].shape, self.weights[i+1].shape, self.gradients[i].shape)
            self.deltas[i] = (self.deltas[i+1].dot(self.weights[i+1].T)) * self.gradients[i]
        
        # update weights
        for i in reversed(range(1, self.nb_layers-1)):
            update = (-self.step_size)*self.outputs[i-1].T.dot(self.deltas[i])
            self.weights[i] += update - (self.lmbda * self.weights[i])
        self.weights[0] += (-self.step_size)*inputs.T.dot(self.deltas[0]) - (self.lmbda * self.weights[0])
        
        # update biases
        for i in range(self.nb_layers-1):
            bias_update = np.sum(self.deltas[i], axis=0) * (-self.step_size)
#             print("bias",self.biases[i].shape, bias_update.shape)
            self.biases[i] = self.biases[i] + bias_update
    
    def train(self, inputs, labels):
        self.fforw(inputs, labels)
        self.backprop(inputs, labels)
        
    def relu(self, x, derivative=False):
        if derivative:
            x[x<=0.0] = 0.0
            x[x>0.0] = 1.0
            return x
        else:
            return x * (x > 0)
    
    def sigmoid(self, x, derivative=False):
        if derivative:
            return x*(1.0-x)
        else:
            return 1.0/(1.0+np.exp(-x))

    def tanh(x, derivative=False):
        if derivative:
            return (1.0 - (np.square(x)))
        else:
            return np.tanh(x)
        
    def fit(self, inputs, labels, epochs):
        t_start = time.clock()
        print("starting fit, time is: ", t_start)
        t_last = time.clock()
        for i in range(epochs):
            print("epoch: ", i + 1)
            # randomize data
            print(np.shape(inputs), np.shape(labels))
            state = np.random.get_state()
            np.random.shuffle(inputs)
            np.random.set_state(state)
            np.random.shuffle(labels)
            print(np.shape(inputs), np.shape(labels))
#             self.update_weights(inputs, labels)
            self.train(inputs, labels)
            t_epoch = time.clock()
            print("time elapsed is: ", t_epoch - t_last)
            
    def predict(self, inp, labels):
        self.fforw(inp, labels)
#         return self.outputs[len(self.nodes) - 2]
#         print("outputs len: ", len(self.outputs))
#         print("outputs", self.outputs[-1])
#         print("outputs", self.outputs[self.nb_layers-2][0])
        return [np.argmax(x) for x in self.outputs[self.nb_layers-2]]
        #output of final layer

# General functions

def safe_divide(num, denom):
    if len(num) != len(denom):
        return []
    return [num[i] / denom[i] if num[i] > 0.0 and denom[i] > 0.0 else 0.0 for i in range(len(num))]

def score(preds, targets, nclasses):
    # calculate fscore for each class, then take macro average (unweighted)
    if (len(preds) != len(targets)):
        return -1.0
    true_pos = [0.0] * nclasses
    pos = [0.0] * nclasses
    rel_pos = [0.0] * nclasses
    for i in range(len(preds)):
        if preds[i] == targets[i]:
            true_pos[int(targets[i])] += 1.0
        pos[int(preds[i])] += 1.0
        rel_pos[int(targets[i])] += 1.0
    
    true_pos = np.array(true_pos)
    pos = np.array(pos)
    rel_pos = np.array(rel_pos)
    print("true_pos", true_pos, "\n", "pos", pos, "\n", "rel_pos", rel_pos, "\n")
    p = safe_divide(true_pos, pos)
    r = safe_divide(true_pos, rel_pos)
    print(p, r)
    
    f_scores = [2.0 * p[i] * r[i] / (p[i] + r[i]) if p[i] + r[i] > 0.0 else 0.0 for i in range(len(p))]
    
    return np.mean(f_scores)

def acc(preds, targets):
    if (len(preds) != len(targets)):
        return -1.0
    correct = 0
    for i in range(len(preds)):
        if preds[i] == targets[i]:
            correct += 1
    return float(correct) / float(len(preds))
            

In [ ]:
start_time = time.clock()
print("Loading train_x.npy, elapsed time: ", time.clock() - start_time)
# x = np.loadtxt("../data/train_x.csv", delimiter=",", dtype=int)
x = np.load("x_centred_train.npy.npy")
print("Loaded train_x.csv. Loading train_y.csv, elapsed time: ", time.clock() - start_time)
y = np.loadtxt("train_y.csv", delimiter=",", dtype=int)
print(y.shape)
x = x.reshape(-1, 64*64)
y = y.reshape(-1, 1)
x[x<1.0] = 0
x[x>1.0] = 1

# print(type(y[0][0]), y[0].shape, y.shape)

# binary_pics = (x > 250) + 0

print(np.shape(x), np.shape(y))
print(x[5000][2053], type(x[5000][2053]))

#shuffle x and y
state = np.random.get_state()
np.random.shuffle(x)
np.random.set_state(state)
np.random.shuffle(y)

#make train/valid split
train_size = 25000
valid_size = 25000
end = train_size + valid_size
train_x = x[0:train_size]
train_y = y[0:train_size]
valid_x = x[train_size: end]
valid_y = y[train_size: end]


In [ ]:
epochs = 50
nhidden = [1, 2, 3]
hiddensize = [1, 5, 50, 500, 2048, 4096]
lmbdas = [0.00001, 0.0001, 0.001, 0.01, 0.1]
learning_rates = [0.0000001, 0.00001, 0.001, 0.1]

def tune(maxepochs, nhidden, hiddensize, lmbdas, learning_rates, train_x, train_y, test_x, test_y):
    res = []
    for lr in learning_rates:
        for lmbda in lmbdas:
            for layers in nhidden:
                for size in hiddensize:
                    nodes = [4096]
                    for layer in range(layers):
                        nodes.append(size)
                    nodes.append(10)
                    NN = Network(nodes, lmbda=lmbda, step_size=lr)
                    for i in range(0, maxepochs):
                        NN.fit(train_x, train_y, 1)
                        preds = NN.predict(test_x, test_y)
                        s = score(preds, test_y, 10)
                        a = acc(preds, test_y)
                        res.append((i, layers, size, lmbda, lr, s, a))
                        if (len(res) % 50 == 0):
                            np.save("nn-res-temp", res)
    return res


# res = tune(epochs, nhidden, hiddensize, lmbdas, learning_rates, train_x, train_y, valid_x, valid_y)
# np.save("nn-res", res)

In [ ]:
# analyze saved res file
res = np.load("nn-res-temp.npy")
res = list(res)
res.sort(key=lambda x : x[5])
print(res[-1])

In [ ]:
# best accuracy
# 2.00000000e+01 1.00000000e+00 5.00000000e+02 1.00000000e-04
#  1.00000000e-07

# best f1-score
#2.20000000e+01 2.00000000e+00 4.09600000e+03 1.00000000e-02
# 1.00000000e-07
print("Training NN, elapsed time: ", time.clock() - start_time)

# best performing model found, training below: :
NN = Network([4096, 4096, 4096, 10], lmbda=1e-2, step_size=1e-7)
NN.fit(train_x, train_y, 5)
print("NN trained. elapsed time: ", time.clock() -start_time)

In [ ]:
#make predictions
print("preds:")
preds = NN.predict(valid_x, valid_y)
print("score:")
print("fscore: ", score(preds, valid_y, 10))
print("acc: ", acc(preds, valid_y))

In [ ]:
# save split data, optional
# np.save("tune_train_x", train_x)
# np.save("tune_train_y", train_y)
# np.save("tune_valid_x", valid_x)
# np.save("tune_valid_y", valid_y)

In [ ]:
print("done")